# 🚦 Analisis Big Data Geospatial - Aksesibilitas Kesehatan dan Transportasi Kota Tangerang dan Kota Medan menggunakan OpenStreetMap dan OSMNX

Proyek mini untuk analisis spasial jaringan jalan dan aksesibilitas kesehatan di Kota Tangerang dan Kota Medan.

## Tujuan:
- Mengambil jaringan jalan dari Open Street Map (OSM)
- Analisis statistik jaringan
- Analisis centrality
- Analisis aksesibilitas ke rumah sakit
- Visualisasi jaringan dan Point-of-Interest (POI)


In [ ]:
# 🔧 Install dependencies
!pip install osmnx --upgrade folium geopandas networkx --quiet

In [ ]:
# 📚 Import libraries
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
# 🌍 Ambil jaringan jalan Kota Tangerang
place1 = "Tangerang, Indonesia"
G = ox.graph_from_place(place1, network_type="drive")
nodes, edges = ox.graph_to_gdfs(G)

# 🌍 Ambil jaringan jalan Kota Medan
place2 = "Medan, Indonesia"
H = ox.graph_from_place(place2, network_type="drive")
nodes, edges = ox.graph_to_gdfs(H)

In [ ]:
# 📊 Statistik jaringan (Kota Tangerang)
stats1 = ox.basic_stats(G)
pd.Series(stats1).head(10)

In [ ]:
# 📊 Statistik jaringan (Kota Medan)
stats2 = ox.basic_stats(H)
pd.Series(stats2).head(10)

In [ ]:
# 📍 Analisis centrality (Kota Tangerang)
degree1 = nx.degree_centrality(G)
nx.set_node_attributes(G, degree1, "degree")
top_nodes1 = sorted(degree1.items(), key=lambda x: x[1], reverse=True)[:5]
top_nodes1

In [ ]:
for node_id, score in top_nodes1:
    node_data = G.nodes[node_id]
    print(f"Simpul: {node_id} | Degree: {score:.6f} | Koordinat: ({node_data['y']}, {node_data['x']})")

In [ ]:
m = folium.Map(location=[node_data['y'], node_data['x']], zoom_start=16)

for node_id, _ in top_nodes1:
    data = G.nodes[node_id]
    folium.Marker(
        location=[data['y'], data['x']],
        popup=f"Node {node_id}",
        icon=folium.Icon(color='red')
    ).add_to(m)

m

In [ ]:
# 📍 Analisis centrality (Kota Medan)
degree2 = nx.degree_centrality(H)
nx.set_node_attributes(G, degree2, "degree")
top_nodes2 = sorted(degree2.items(), key=lambda x: x[1], reverse=True)[:5]
top_nodes2

In [ ]:
for node_id, score in top_nodes2:
    node_data = H.nodes[node_id]
    print(f"Simpul: {node_id} | Degree: {score:.6f} | Koordinat: ({node_data['y']}, {node_data['x']})")

In [ ]:
m = folium.Map(location=[node_data['y'], node_data['x']], zoom_start=16)

for node_id, _ in top_nodes2:
    data = H.nodes[node_id]
    folium.Marker(
        location=[data['y'], data['x']],
        popup=f"Node {node_id}",
        icon=folium.Icon(color='red')
    ).add_to(m)

m

In [ ]:
# 🗺️ Visualisasi jaringan (Kota Tangerang)
fig, ax = ox.plot_graph(G, node_size=5, edge_color="gray")

In [ ]:
# 🗺️ Visualisasi jaringan (Kota Medan)
fig, ax = ox.plot_graph(H, node_size=5, edge_color="gray")

In [ ]:
# 🏥 Ambil data rumah sakit dari OSM (Kota Tangerang)
tags = {'amenity': 'hospital'}
hospitals1 = ox.features_from_place(place1, tags=tags)
hospital_points1 = hospitals1.centroid.to_frame(name='geometry').reset_index()
hospital_points1 = gpd.GeoDataFrame(hospital_points1, geometry='geometry', crs=hospitals1.crs)
hospital_points1.head()

In [ ]:
# 🏥 Ambil data rumah sakit dari OSM (Kota Medan)
tags = {'amenity': 'hospital'}
hospitals2 = ox.features_from_place(place2, tags=tags)
hospital_points2 = hospitals2.centroid.to_frame(name='geometry').reset_index()
hospital_points2 = gpd.GeoDataFrame(hospital_points2, geometry='geometry', crs=hospitals1.crs)
hospital_points2.head()

In [ ]:
# 📐 Analisis jarak dari rumah sakit ke node terdekat (Kota Tangerang)
hospital_points1['nearest_node'] = hospital_points1['geometry'].apply(
    lambda point: ox.distance.nearest_nodes(G, point.x, point.y))

source_node = hospital_points1['nearest_node'].iloc[0]
lengths = nx.single_source_dijkstra_path_length(G, source_node, weight='length')

pd.Series(lengths).plot(kind='hist', bins=30, title='Distribusi Jarak dari RS ke Semua Titik')

In [ ]:
# 📐 Analisis jarak dari rumah sakit ke node terdekat (Kota Medan)
hospital_points2['nearest_node'] = hospital_points2['geometry'].apply(
    lambda point: ox.distance.nearest_nodes(H, point.x, point.y))

source_node = hospital_points2['nearest_node'].iloc[0]
lengths = nx.single_source_dijkstra_path_length(H, source_node, weight='length')

pd.Series(lengths).plot(kind='hist', bins=30, title='Distribusi Jarak dari RS ke Semua Titik')

In [ ]:
# 🗺️ Peta interaktif jaringan dan rumah sakit (Kota Tangerang)
m = folium.Map(location=[-6.1783, 106.6319], zoom_start=12)
for idx, row in hospital_points1.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup="Rumah Sakit").add_to(m)
folium.GeoJson(edges).add_to(m)
m

In [ ]:
# 🗺️ Peta interaktif jaringan dan rumah sakit (Kota Medan)
m = folium.Map(location=[3.5952, 98.6722], zoom_start=12)
for idx, row in hospital_points2.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup="Rumah Sakit").add_to(m)
folium.GeoJson(edges).add_to(m)
m

**(Ditambahkan) - Akes ke Point of Interest**

Akan dilakukan analisis pada rumah sakit pada entri ke - [0]

Ditampilkan visualisasi heatmap dan buffer + colormap

In [ ]:
## KOTA TANGERANG ##

# Ambil node rumah sakit sumber dari baris pertama
source_node = hospital_points1['nearest_node'].iloc[0]

# Hitung jarak dijkstra dari node rumah sakit ke semua node lain
lengths = nx.single_source_dijkstra_path_length(G, source_node, weight='length')

# Konversi ke DataFrame
distance_df = pd.DataFrame.from_dict(lengths, orient='index', columns=['distance_to_hospital'])

###

from folium.plugins import HeatMap

# Ambil koordinat node
node_xy = {node: (data['y'], data['x']) for node, data in G.nodes(data=True)}

# Buat data heatmap: latitude, longitude, "intensitas" aksesibilitas (semakin kecil jarak, semakin panas)
heat_data = [
    [node_xy[node][0], node_xy[node][1], 1 / (dist + 1)]  # tambah 1 agar tidak div 0
    for node, dist in lengths.items()
]

# Peta interaktif
m = folium.Map(location=[hospital_points1.geometry.y.iloc[0], hospital_points1.geometry.x.iloc[0]], zoom_start=13)
HeatMap(heat_data, radius=10, blur=5, max_zoom=16).add_to(m)

# Tambahkan marker rumah sakit
folium.Marker(
    location=[hospital_points1.geometry.y.iloc[0], hospital_points1.geometry.x.iloc[0]],
    popup='Rumah Sakit',
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

m  # Menampilkan peta

In [ ]:
## KOTA MEDAN ##

# Ambil node rumah sakit sumber dari baris pertama
source_node = hospital_points2['nearest_node'].iloc[0]

# Hitung jarak dijkstra dari node rumah sakit ke semua node lain
lengths = nx.single_source_dijkstra_path_length(H, source_node, weight='length')

# Konversi ke DataFrame
distance_df = pd.DataFrame.from_dict(lengths, orient='index', columns=['distance_to_hospital'])

###

from folium.plugins import HeatMap

# Ambil koordinat node
node_xy = {node: (data['y'], data['x']) for node, data in H.nodes(data=True)}

# Buat data heatmap: latitude, longitude, "intensitas" aksesibilitas (semakin kecil jarak, semakin panas)
heat_data = [
    [node_xy[node][0], node_xy[node][1], 1 / (dist + 1)]  # tambah 1 agar tidak div 0
    for node, dist in lengths.items()
]

# Peta interaktif
m = folium.Map(location=[hospital_points2.geometry.y.iloc[0], hospital_points2.geometry.x.iloc[0]], zoom_start=13)
HeatMap(heat_data, radius=10, blur=5, max_zoom=16).add_to(m)

# Tambahkan marker rumah sakit
folium.Marker(
    location=[hospital_points2.geometry.y.iloc[0], hospital_points2.geometry.x.iloc[0]],
    popup='Rumah Sakit',
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

m  # Menampilkan peta

In [ ]:
## KOTA TANGERANG ##

# Proyeksikan graph ke CRS metrik (contoh: EPSG:32648 untuk zona UTM Jakarta)
G_proj = ox.project_graph(G, to_crs="EPSG:32648")

# Proyeksikan titik rumah sakit ke CRS metrik juga
hospital_points_proj = hospital_points1.to_crs("EPSG:32648")

# Ambil node terdekat dari rumah sakit pertama
source_node = ox.distance.nearest_nodes(
    G_proj,
    X=hospital_points_proj.geometry.iloc[0].x,
    Y=hospital_points_proj.geometry.iloc[0].y
)

# Hitung jarak dari rumah sakit ke semua node di jaringan jalan
lengths = nx.single_source_dijkstra_path_length(G_proj, source_node, weight='length')

# Konversi graph ke GeoDataFrame dan tambahkan jarak sebagai atribut
nodes = ox.graph_to_gdfs(G_proj, edges=False)
nodes['distance_to_hospital'] = nodes.index.map(lengths)

# Buat buffer 1 km di sekitar rumah sakit
hospital_buffer = hospital_points_proj.iloc[[0]].copy()
hospital_buffer['geometry'] = hospital_buffer.buffer(1000)

# Plot hasil
fig, ax = plt.subplots(figsize=(10, 10))
nodes.plot(column='distance_to_hospital', cmap='viridis', markersize=5, legend=True, ax=ax)
hospital_buffer.plot(ax=ax, color='red', alpha=0.2, label='Buffer 1km')
hospital_points_proj.iloc[[0]].plot(ax=ax, color='black', markersize=50, label='Rumah Sakit')
plt.title('Aksesibilitas ke Rumah Sakit [0] - Jarak ke Node Jalan')
plt.legend()
plt.show()

In [ ]:
## KOTA MEDAN ##

# GeoDataFrame node jalan
nodes = ox.graph_to_gdfs(H, edges=False)
nodes['distance_to_hospital'] = nodes.index.map(lengths)

# Buffer 1km di sekitar rumah sakit
hospital_buffer = hospital_points2.iloc[[0]].copy()
hospital_buffer = hospital_buffer.to_crs(epsg=32648)  # CRS metrik, contoh: UTM zona Jakarta
hospital_buffer['geometry'] = hospital_buffer.buffer(1000)

# Ubah nodes ke CRS yang sama agar bisa diplot
nodes_proj = nodes.to_crs(hospital_buffer.crs)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
nodes_proj.plot(column='distance_to_hospital', cmap='viridis', markersize=5, legend=True, ax=ax)
hospital_buffer.plot(ax=ax, color='red', alpha=0.2, label='Buffer 1km')
hospital_points2.iloc[[0]].to_crs(hospital_buffer.crs).plot(ax=ax, color='black', markersize=50, label='Rumah Sakit')
plt.title('Aksesibilitas ke Rumah Sakit [0] - Jarak ke Node Jalan')
plt.legend()
plt.show()

**Visualisasi jaringan dengan node berwarna menurut degree centrality**

In [ ]:
## KOTA TANGERANG ##

# Ekstrak nilai degree centrality sesuai urutan node
node_color = [degree1[node] for node in G.nodes()]

# 3. Visualisasi dengan warna berdasarkan centrality
fig, ax = ox.plot_graph(
    G,
    node_color=node_color,
    node_size=20,
    edge_color='#999999',
    edge_linewidth=0.5,
    bgcolor='white',
    node_zorder=2,
    show=False,
    close=False
)

# 4. Tambahkan colorbar
sm = plt.cm.ScalarMappable(cmap='plasma', norm=plt.Normalize(vmin=min(node_color), vmax=max(node_color)))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.6)
cbar.set_label('Degree Centrality')

plt.title("Jaringan Jalan - Warna Node Berdasarkan Degree Centrality", fontsize=12)
plt.show()

In [ ]:
## KOTA MEDAN ##

# Ekstrak nilai degree centrality sesuai urutan node
node_color = [degree2[node] for node in H.nodes()]

# 3. Visualisasi dengan warna berdasarkan centrality
fig, ax = ox.plot_graph(
    H,
    node_color=node_color,
    node_size=20,
    edge_color='#999999',
    edge_linewidth=0.5,
    bgcolor='white',
    node_zorder=2,
    show=False,
    close=False
)

# 4. Tambahkan colorbar
sm = plt.cm.ScalarMappable(cmap='plasma', norm=plt.Normalize(vmin=min(node_color), vmax=max(node_color)))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.6)
cbar.set_label('Degree Centrality')

plt.title("Jaringan Jalan - Warna Node Berdasarkan Degree Centrality", fontsize=12)
plt.show()